## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import ClassicalRegister, AncillaRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

from LogicalQ.LogicalGeneral import LogicalCircuitGeneral
from LogicalQ.Library.QECCs import five_qubit_code, steane_code, four_qubit_code, shor_code
from LogicalQ.NoiseModel import construct_noise_model_QuantinuumH1_1
from LogicalQ.Experiments import execute_circuits

import random

%load_ext autoreload
%autoreload 2

## Demonstrating error correction with 5-qubit code

In [ ]:
code = five_qubit_code
n = code['label'][0]

In [ ]:
#Tests errors measuring in Z basis
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,1])
    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'X') #Also try testing Z type errors
        five_qubit_circ.add_error(1,i,'X')
    five_qubit_circ.append_qec_cycle([0,1])
    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z')
    simulator = AerSimulator()
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

In [ ]:
five_qubit_circ = LogicalCircuitGeneral(2, **code)
five_qubit_circ.encode(0,1, initial_states=[0,1])
five_qubit_circ.add_error(0,1,'X')
five_qubit_circ.add_error(1,1,'X')
five_qubit_circ.append_qec_cycle([0,1])
five_qubit_circ.measure([0,1], [0,1], meas_basis='Z')
five_qubit_circ.draw(output='mpl')

In [ ]:
#Tests errors measuring in X basis
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,1])

    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'Z') #Also try testing Z type errors
        five_qubit_circ.add_error(1,i,'Z')
    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.h([0,1])

    five_qubit_circ.measure([0,1], [0,1], meas_basis='X')
    
    simulator = AerSimulator()
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

In [ ]:
#Tests errors measuring in Y basis
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,1])
    
    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'X') #Also try testing Z type errors
        five_qubit_circ.add_error(1,i,'X')
    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.h([0,1])
    five_qubit_circ.s([0,1])
    
    five_qubit_circ.measure([0,1], [0,1], meas_basis='Y', with_error_correction=True)
    simulator = AerSimulator()
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

In [ ]:
#Tests CX pauli frame updates
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[1,1])

    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(1,i,'X') #Also try testing Z type errors

    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.cx(1,0)

    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z', with_error_correction=True)
    simulator = AerSimulator(method='stabilizer')
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

In [ ]:
#Tests CZ pauli frame updates
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,0])

    five_qubit_circ.h(1)
    five_qubit_circ.cx(1,0)

    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(1,i,'X') #Also try testing Z type errors

    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.cz(1,0)
    five_qubit_circ.cx(1,0)
    five_qubit_circ.h(1)

    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z', with_error_correction=True)
    simulator = AerSimulator(method='stabilizer')
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

In [ ]:
#Tests CY pauli frame updates
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,0])

    five_qubit_circ.h(1)
    
    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'X') #Also try testing Z type errors

    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.cy(1,0)
    five_qubit_circ.s(1)

    five_qubit_circ.cx(1,0)
    five_qubit_circ.h(1)

    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z', with_error_correction=True)
    simulator = AerSimulator(method='stabilizer')
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

In [ ]:
#Tests T and T^dagger gate Pauli updates
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,1])
    five_qubit_circ.h([0,1])
    five_qubit_circ.t([0,1])

    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'X') #Also try testing Z type errors
        five_qubit_circ.add_error(1,i,'X')
    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.s([0,1])
    five_qubit_circ.tdg([0,1])
    five_qubit_circ.sdg([0,1])

    five_qubit_circ.h([0,1])

    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z')
    simulator = AerSimulator()
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

## Demonstrating error correction with Shor code

In [ ]:
code = shor_code
n = code['label'][0]

In [ ]:
shor_circ = LogicalCircuitGeneral(1, **code)
shor_circ.encode(0, initial_states=[0])
shor_circ.draw(output='mpl')

### Verify Shor encoding

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

def remove_past_barrier(circuit: QuantumCircuit) -> QuantumCircuit:
    """
    Extracts all operations from a quantum circuit that occur before the first barrier.

    Args:
        circuit: The input Qiskit QuantumCircuit.

    Returns:
        A new QuantumCircuit containing only the operations before the first barrier.
        The new circuit will have the same quantum and classical registers as the original.
    """
    new_circuit = QuantumCircuit(*circuit.qregs, *circuit.cregs, name="pre-barrier section")

    for instruction in circuit.data:
        op = instruction.operation
        
        if op.name == 'barrier':
            break
        
        new_circuit.append(instruction)
            
    return new_circuit

def prune_circ(circuit: QuantumCircuit, num_qubits_to_keep: int) -> QuantumCircuit:
    """
    Recreates circuit with only first N qubits.
    """
    new_circuit = QuantumCircuit(num_qubits_to_keep, name=f"first_{num_qubits_to_keep}_qubits")
    for instruction in circuit.data:
        qubit_indices = [circuit.qubits.index(q) for q in instruction.qubits]
        if not qubit_indices or max(qubit_indices) < num_qubits_to_keep:
            new_qubits = [new_circuit.qubits[i] for i in qubit_indices]
            new_circuit.append(instruction.operation, new_qubits, instruction.clbits)

    return new_circuit

def measure_stabilizer(qc: QuantumCircuit, G, idx: int):
    """Measure the idx'th stabilizer in the list"""
    #n,k,d = qecc["label"]
    #stabilizer_tableau = qecc["stabilizer_tableau"]
    #stabilizer = stabilizer_tableau[idx]

    # Add ancilla for stabilizer measurement
    a_reg = AncillaRegister(1)
    qc.add_register(a_reg)
    c_reg = ClassicalRegister(1)
    qc.add_register(c_reg)
    
    # Perform measurement
    qc.barrier()
    qc.h(qc.ancillas[0])
    for i in range(9):
        perform_X = G[0, idx, i]
        perform_Z = G[1, idx, i]
        if perform_X == 1:
            qc.cx(qc.ancillas[0], qc.qubits[i])
        elif perform_Z == 1:
            qc.cz(qc.ancillas[0], qc.qubits[i])
    qc.h(qc.ancillas[0])
    qc.measure(qc.ancillas[0], qc.clbits[0])
    qc.barrier()
    return qc
    
def generate_encoding(message: int, stabilizer: int, error=None, error_target=None):
    shor_circ = LogicalCircuitGeneral(1, **shor_code)
    G = shor_circ.G.copy() # Extract stabilizer matrix for stabilizer measurement before pruning circuit
    shor_circ.encode(0, initial_states=[message])
    shor_circ = prune_circ(remove_past_barrier(shor_circ), 9)
    
    # Apply error
    if error_target is None:
        error_target = random.randint(0, 8)
    if error == 'X':
        shor_circ.x(error_target)
    elif error == 'Z':
        shor_circ.z(error_target)
    
    shor_circ = measure_stabilizer(shor_circ, G, stabilizer)
    return shor_circ

"""
for stabilizer in range(8):
    outputs = []
    for i in range(100):
        qc = generate_encoding(0, 2, error=None)
        simulator = AerSimulator()
        result = execute_circuits(qc, backend=simulator, shots=1, memory=True)[0]
        outputs.append(int(result.get_memory()[0]))
    outputs = np.array(outputs)
    print(np.mean(outputs))
"""

In [ ]:
from tqdm.notebook import tqdm
shor_circ = LogicalCircuitGeneral(1, **shor_code)
G = shor_circ.G.copy()
G_shape = list(G.shape)
G_shape[0] += 1
del shor_circ
measured_errors = np.zeros([2] + G_shape)

for k,message in tqdm(enumerate([0,1]), position=0, leave=False):
    for j,error in tqdm(enumerate(['X', 'Z', None]), position=1, leave=False):
        for stabilizer in tqdm(range(8), position=2, leave=False):
            for error_target in tqdm(range(9), position=3, leave=False):
                qc = generate_encoding(message, stabilizer, error=error, error_target=error_target)
                simulator = AerSimulator()
                result = execute_circuits(qc, backend=simulator, shots=1, memory=True)[0]
                syndrome_bit = int(result.get_memory()[0])
                measured_errors[k, j, stabilizer, error_target] = syndrome_bit

Check that stabilizers of G matrix produced by LogicalQ as a simplification of the original tableau is still valid (i.e. all stabilizers commute)

In [ ]:
np.mod(G[0] @ G[1].T + G[1] @ G[0].T, 2)

Plot syndrome measurements against generator matrix.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

fig, axs = plt.subplots(3, 2, figsize=(8, 10))

m=1

axs[0, 0].imshow(G[0], cmap='binary', interpolation='none')
axs[0, 0].set_title("Generator G[0] (X comp.)")

axs[0, 1].imshow(measured_errors[m,0], cmap='binary', interpolation='none')
axs[0, 1].set_title("Syndrome (from X error)")
axs[0, 1].set_aspect('auto')


axs[1, 0].imshow(G[1], cmap='binary', interpolation='none')
axs[1, 0].set_title("Generator G[1] (Z comp.)")


axs[1, 1].imshow(measured_errors[m,1], cmap='binary', interpolation='none')
axs[1, 1].set_title("Syndrome (from Z error)")
axs[1, 1].set_aspect('auto')


axs[2, 1].imshow(measured_errors[m,2], cmap='binary', interpolation='none')
axs[2, 1].set_title("Syndrome (No error)")
axs[2, 1].set_aspect('auto')

axs[2, 0].axis('off')

axs[0, 0].set_ylabel("Stabilizer Index")
axs[1, 0].set_ylabel("Stabilizer Index")

axs[1, 0].set_xlabel("Qubit Index")
axs[2, 1].set_xlabel("Stabilizer Index")

white_patch = mpatches.Patch(color='white', label='+1 / 0', ec='black')
black_patch = mpatches.Patch(color='black', label='-1 / 1')

legend1 = fig.legend(
    handles=[white_patch, black_patch],
    labels=['+1 (No Error)', '-1 (Error Detected)'],
    title_fontsize='large',
    loc='center left',
    bbox_to_anchor=(0.15, 0.16),
    title="Syndrome Eigenvalues"
)

fig.add_artist(legend1)

legend2 = fig.legend(
    handles=[white_patch, black_patch],
    labels=['0', '1'],
    title_fontsize='large',
    loc='center left',
    bbox_to_anchor=(0.15, 0.26),
    title="Generator Values"
)

#plt.tight_layout(rect=[0, 0.08, 1, 1]) # rect=[left, bottom, right, top]

plt.show()

In [ ]:
shor_circ = LogicalCircuitGeneral(1, **shor_code)

In [ ]:
qc.draw(output='mpl')

### Looking at circuit structure

In [ ]:
shor_circ = LogicalCircuitGeneral(1, **shor_code)
shor_circ.encode(0, initial_states=[0])
shor_circ.append_qec_cycle([0])
shor_circ.measure([0], [0])
shor_circ.draw(output='mpl')

### Full QEC cycle testing

In [ ]:
code = shor_code
n=9

In [58]:
#Tests errors measuring in Z basis
outputs = []
for i in range(n+1):
    shor_circ = LogicalCircuitGeneral(1, **code)
    shor_circ.encode(0, initial_states=[0])
    #if i < n: #Runs the last sim without an error for thorough testing
        #shor_circ.add_error(0,i,'X') #Also try testing Z type errors
        #shor_circ.add_error(1,i,'X')
    shor_circ.append_qec_cycle([0])
    shor_circ.measure([0], [0], meas_basis='Z')
    simulator = AerSimulator()
    result = execute_circuits(shor_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '0'
for o in outputs:
    print(o)

['1 111111000 00 00 10000000 10000000 10000000 0']
['0 000000000 00 00 00000000 00000000 00000000 0']
['0 000000000 00 00 10000000 10000000 10000000 0']
['0 000111111 00 00 10000000 10000000 10000000 0']
['0 000000000 00 00 00000000 00000000 00000000 0']
['1 111111000 00 00 10000000 10000000 10000000 0']
['0 000000000 00 00 10000000 10000000 10000000 0']
['1 000110101 00 00 00000111 00000111 00000111 0']
['1 111000111 00 00 10000000 10000000 10000000 0']
['1 111000111 00 00 10000000 10000000 10000000 0']


In [ ]:
shor_circ.draw(output='mpl')